In [1]:
import re
import time
import pytz
import calendar
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
import Chatbot_Entity_date as ed

import nltk
import spacy
from konlpy.tag import Okt, Kkma
from spacy.tokens import Span
from pykospacing import Spacing
from soynlp.normalizer import repeat_normalize
from typing import List, Dict, Optional, Tuple, Set, Union

okt = Okt()
kkma = Kkma()
spacing = Spacing()
kst = pytz.timezone('Asia/Seoul')  # 한국 표준시(KST) 타임존 설정

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\4\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\4\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\4\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\4\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
# 텍스트 전처리
def processe_text(text):
    text = spacing(text)
    text = re.sub(r"[^가-힣a-zA-Z0-9\s]", "", text)
    text = repeat_normalize(text, num_repeats=3)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [3]:
# 의도=금융 엔티티 추출 함수

def extract_finance_entities(text):

    # 금융 관련 패턴
    patterns = {
        "EXPENDITURE": r"지출|소비|쓴|사용|결제|카드",
        "INCOME": r"수입|소득|월급|급여",
        "BUDGET": r"예산",
        "LOAN": r"대출",
        "DEPOSIT_SAVINGS": r"적금|예금|저축",
        "TRANSACTION": r"입출금|입금|출금|이체|송금|인출|납부",
        "ASSET" : r"보유|자산|재정|재무|자본|재산|잔고",
        "STOCK" : r"주식|삼성|삼전|애플|코인|비트코인|samsung|apple|coin|bitcoin",
        "buy" : r"구매|구입|매입|매수|투자|\b산\b",
        "sell" : r"판매|매도|\b판\b|처분",
        "ALL": r"가계부|가계|금전",
    }

    # 표현 패턴
    patterns2 = {
        "stats" : r"비교|통계|보고|정리|분석|현황",  # 내역+합계
        "simple" : r"내역|상황|항목|목록|기록|출처|조회|정보|사항|이력|내용",  # 내역
        "sum" : r"합계|총액|잔액|잔고|총합|누적|합산|총계|전체금액|최종금액",  # 합계
        "average" : r"평균",  # 평균
        "date" : r"언제",  # 날짜
        "sort" : r"가장|큰|작은|제일|많이|적게|높은|낮은|순위|순서|자주|반복|빈번|주요|적은|많은",  # 정렬
    }

    # 패턴에 맞는 주요 키워드 추출 함수
    def extract_main_keyword(text, pattern):
        match = re.search(pattern, text)
        if match:
            return match.group(0)
        return None

    # 텍스트에서 조사를 제거하는 함수
    def clean_text(text):
        cleaned_text = re.sub(r'(과|와|의|가|이|을|를|은|는|에서|으로|고|까지|부터|도|만|조차|뿐|에|와|에서|로)$', '', text)
        return cleaned_text

    # 사용자 정의 spaCy 파이프라인 컴포넌트
    @spacy.Language.component("custom_finance_entity_adder")
    def custom_finance_entity_adder(doc):
        
        new_ents = []
        for token in doc:
            noun_phrase = ''.join([word for word, tag in okt.pos(token.text) if tag in ['Noun', 'Alpha']])
            noun_phrase_cleaned = clean_text(noun_phrase)
            original_text_cleaned = clean_text(token.text)

            found_pattern1 = False
            found_pattern2 = False

            # 패턴1 매칭
            for label, pattern in patterns.items():
                main_keyword = extract_main_keyword(noun_phrase_cleaned, pattern)
                if main_keyword:
                    new_ent = Span(doc, token.i, token.i + 1, label=f"{label}_pattern1")
                    new_ent._.set("cleaned_text", noun_phrase_cleaned)
                    new_ents.append(new_ent)
                    found_pattern1 = True
                    break

            if not found_pattern1:
                for label, pattern in patterns2.items():
                    main_keyword = extract_main_keyword(noun_phrase_cleaned, pattern)
                    if main_keyword:
                        new_ent = Span(doc, token.i, token.i + 1, label=f"{label}_pattern2")
                        new_ent._.set("cleaned_text", noun_phrase_cleaned)
                        new_ents.append(new_ent)
                        found_pattern2 = True
                        break

            # 형태소 분석 결과에 매칭되지 않은 경우, 원래 텍스트로 매칭 시도
            if not found_pattern1 and not found_pattern2:
                for label, pattern in patterns.items():
                    main_keyword = extract_main_keyword(original_text_cleaned, pattern)
                    if main_keyword:
                        new_ent = Span(doc, token.i, token.i + 1, label=f"{label}_pattern1")
                        new_ent._.set("cleaned_text", original_text_cleaned)
                        new_ents.append(new_ent)
                        break

            if not found_pattern1 and not found_pattern2:
                for label, pattern in patterns2.items():
                    main_keyword = extract_main_keyword(original_text_cleaned, pattern)
                    if main_keyword:
                        new_ent = Span(doc, token.i, token.i + 1, label=f"{label}_pattern2")
                        new_ent._.set("cleaned_text", original_text_cleaned)
                        new_ents.append(new_ent)
                        break

        doc.ents = new_ents

        return doc

    # spaCy 모델 로드
    nlp = spacy.load("ko_core_news_sm")

    # 확장 속성 등록 (cleaned_text)
    Span.set_extension("cleaned_text", default=None, force=True)

    # custom_finance_entity_adder가 이미 파이프라인에 존재하면 제거
    if "custom_finance_entity_adder" in nlp.pipe_names:
        nlp.remove_pipe("custom_finance_entity_adder")

    # custom_finance_entity_adder를 spaCy 파이프라인에 추가
    nlp.add_pipe("custom_finance_entity_adder", after="ner")

    # 텍스트 처리
    doc = nlp(text)

    # 결과 출력
    entities = {"pattern1": [], "pattern2": []}
    
    for ent in doc.ents:
        if "_pattern1" in ent.label_:
            entities["pattern1"].append((ent._.get("cleaned_text"), ent.label_.replace("_pattern1", "")))
        elif "_pattern2" in ent.label_:
            entities["pattern2"].append((ent._.get("cleaned_text"), ent.label_.replace("_pattern2", "")))
        
    if not entities['pattern2']:
        entities['pattern2'].append(('기본값', 'simple'))
    return entities


In [4]:
# 날짜 쿼리 추출을 위한 함수

# 소비, 수입, 입출금 등을 위한 패턴
def process_date_format(input_date):
    if input_date == None:
        input_date = [datetime.strftime(datetime.today(), '%Y-%m-%d')]  # 오늘

    if len(input_date[0]) == 10:  # 형식이 'yyyy-mm-dd'
        date_query = "AND rp_date IN ("
        for i in input_date:
            date_query += f'"{i}", '
        date_query = date_query.rstrip()
        date_query = date_query[:-1] + ")"
        return date_query

    elif len(input_date[0]) == 7:  # 형식이 'yyyy-mm'
        date_query = "AND "
        for i in input_date:
            date_query += f'rp_date LIKE "{i}%" OR '
        date_query = date_query[:-4]
        # year, month = map(int, input_date[0].split('-'))
        # start_date = datetime(year, month, 1).strftime('%Y-%m-%d')  # 그 달의 1일
        # last_day = calendar.monthrange(year, month)[1]  # 그 달의 마지막 날 계산
        # end_date = datetime(year, month, last_day).strftime('%Y-%m-%d')  # 그 달의 마지막 날
        # date_query = f"AND rp_date BETWEEN '{start_date}' AND '{end_date}'"
        return date_query

    elif len(input_date[0]) == 4:  # 형식이 'yyyy'
        date_query = "AND "
        for i in input_date:
            date_query += f'rp_date LIKE "{i}%" OR '
        date_query = date_query[:-4]
        # year = int(input_date[0])
        # start_date = datetime(year, 1, 1).strftime('%Y-%m-%d')  # 그 해의 1월 1일
        # end_date = datetime(year, 12, 31).strftime('%Y-%m-%d')  # 그 해의 12월 31일
        # date_query = f"AND rp_date BETWEEN '{start_date}' AND '{end_date}'"
        return date_query
    else:
        raise ValueError("잘못된 날짜 형식입니다.")


def process_month_format(input_date):
    if input_date == None:
        input_date = [datetime.strftime(datetime.today(), '%Y-%m')]  # 이번달
        
    if len(input_date[0]) == 10:  # 형식이 'yyyy-mm-dd'
        date_query = "AND rp_date IN ("
        for i in input_date:
            date_query += f'"{i}", '
        date_query = date_query.rstrip()
        date_query = date_query[:-1] + ")"
        return date_query

    elif len(input_date[0]) == 7:  # 형식이 'yyyy-mm'
        date_query = "AND "
        for i in input_date:
            date_query += f'rp_date LIKE "{i}%" OR '
        date_query = date_query[:-4]
        return date_query

    elif len(input_date[0]) == 4:  # 형식이 'yyyy'
        date_query = "AND "
        for i in input_date:
            date_query += f'rp_date LIKE "{i}%" OR '
        date_query = date_query[:-4]
        return date_query
    else:
        raise ValueError("잘못된 날짜 형식입니다.")


def process_all_format(input_date):
    if input_date == None:
        date_query = ""
        return date_query
        
    elif len(input_date[0]) == 10:  # 형식이 'yyyy-mm-dd'
        date_query = "AND rp_date IN ("
        for i in input_date:
            date_query += f'"{i}", '
        date_query = date_query.rstrip()
        date_query = date_query[:-1] + ")"
        return date_query

    elif len(input_date[0]) == 7:  # 형식이 'yyyy-mm'
        date_query = "AND "
        for i in input_date:
            date_query += f'rp_date LIKE "{i}%" OR '
        date_query = date_query[:-4]
        return date_query

    elif len(input_date[0]) == 4:  # 형식이 'yyyy'
        date_query = "AND "
        for i in input_date:
            date_query += f'rp_date LIKE "{i}%" OR '
        date_query = date_query[:-4]
        return date_query
    else:
        raise ValueError("잘못된 날짜 형식입니다.")


# 주식 buy/sell을 위한 패턴
def process_date_format_stock(input_date):
    if input_date == None:
        input_date = [datetime.strftime(datetime.today(), '%Y-%m-%d')]  # 오늘

    if len(input_date[0]) == 10:  # 형식이 'yyyy-mm-dd'
        date_query = "AND sh_date IN ("
        for i in input_date:
            date_query += f'"{i}", '
        date_query = date_query.rstrip()
        date_query = date_query[:-1] + ")"
        return date_query

    elif len(input_date[0]) == 7:  # 형식이 'yyyy-mm'
        date_query = "AND "
        for i in input_date:
            date_query += f'rp_date LIKE "{i}%" OR '
        date_query = date_query[:-4]
        # year, month = map(int, input_date[0].split('-'))
        # start_date = datetime(year, month, 1).strftime('%Y-%m-%d')  # 그 달의 1일
        # last_day = calendar.monthrange(year, month)[1]  # 그 달의 마지막 날 계산
        # end_date = datetime(year, month, last_day).strftime('%Y-%m-%d')  # 그 달의 마지막 날
        # date_query = f"AND sh_date BETWEEN '{start_date}' AND '{end_date}'"
        return date_query

    elif len(input_date[0]) == 4:  # 형식이 'yyyy'
        date_query = "AND "
        for i in input_date:
            date_query += f'rp_date LIKE "{i}%" OR '
        date_query = date_query[:-4]
        # year = int(input_date[0])
        # start_date = datetime(year, 1, 1).strftime('%Y-%m-%d')  # 그 해의 1월 1일
        # end_date = datetime(year, 12, 31).strftime('%Y-%m-%d')  # 그 해의 12월 31일
        # date_query = f"AND sh_date BETWEEN '{start_date}' AND '{end_date}'"
        return date_query
    else:
        raise ValueError("잘못된 날짜 형식입니다.")


# 주식 수량을 위한 패턴
def process_date_format_stock_qty(input_date):
    if input_date == None:
        input_date = [datetime.strftime(datetime.today(), '%Y-%m-%d')]  # 오늘

    if len(input_date[0]) == 10:  # 형식이 'yyyy-mm-dd'
        date_query = f"AND sh_date <= '{input_date[0]}'"
        return date_query

    elif len(input_date[0]) == 7:  # 형식이 'yyyy-mm'
        year, month = map(int, input_date[0].split('-'))
        start_date = datetime(year, month, 1).strftime('%Y-%m-%d')  # 그 달의 1일
        last_day = calendar.monthrange(year, month)[1]  # 그 달의 마지막 날 계산
        end_date = datetime(year, month, last_day).strftime('%Y-%m-%d')  # 그 달의 마지막 날
        date_query = f"AND sh_date <= '{end_date}'"
        return date_query

    elif len(input_date[0]) == 4:  # 형식이 'yyyy'
        year = int(input_date[0])
        start_date = datetime(year, 1, 1).strftime('%Y-%m-%d')  # 그 해의 1월 1일
        end_date = datetime(year, 12, 31).strftime('%Y-%m-%d')  # 그 해의 12월 31일
        date_query = f"AND sh_date <= '{end_date}'"
        return date_query
    else:
        raise ValueError("잘못된 날짜 형식입니다.")


# 예산을 위한 패턴
def find_month(time):
    try:
        if '월' in time and '년' not in time:  # '8월'과 같은 형식
            current_year = datetime.today().year
            input_time = datetime.strptime(f'{current_year}-{time[:-1]}', '%Y-%m')
            return input_time
        elif '년' in time and '월' in time:  # '2024년 8월'과 같은 형식
            input_time = datetime.strptime(time.replace('년 ', '-').replace('월', ''), '%Y-%m')
            return input_time
        elif len(time) == 7 and '-' in time:  # '2024-08'과 같은 형식
            input_time = datetime.strptime(time, '%Y-%m')
            return input_time
        elif len(time) == 10 and '-' in time:  # '2024-01-01'과 같은 형식
            input_time = datetime.strptime(time, '%Y-%m-%d')
            return input_time
        elif '년' in time and '월' not in time and '일' not in time:  # '2024년'과 같은 형식
            input_time = datetime.strptime(time.replace('년', ''), '%Y')
            return input_time
    except ValueError:
        return "잘못된 날짜 형식입니다."


# 날짜 형식에 따른
def parse_input_date(date_str):
    if len(date_str) == 4:  # 'YYYY' 형식
        input_date_vs = datetime.strptime(date_str, '%Y')
        today = datetime.now()  # 현재 연도만 사용
        today = datetime.strptime(today.strftime('%Y'), '%Y')  # 연도만 비교
        return input_date_vs, today
    elif len(date_str) == 7:  # 'YYYY-MM' 형식
        input_date_vs = datetime.strptime(date_str, '%Y-%m')
        today = datetime.now()  # 현재 연도-월만 사용
        today = datetime.strptime(today.strftime('%Y-%m'), '%Y-%m')  # 연도-월만 비교
        return input_date_vs, today
    elif len(date_str) == 10:  # 'YYYY-MM-DD' 형식
        input_date_vs = datetime.strptime(date_str, '%Y-%m-%d')
        today = datetime.now()  # 현재 연도-월-일 사용
        today = datetime.strptime(today.strftime('%Y-%m-%d'), '%Y-%m-%d')  # 연도-월-일 비교
        return input_date_vs, today
    else:
        raise ValueError("지원되지 않는 날짜 형식입니다.")


# 원래의 날짜 형식으로 되돌리는 함수
def format_back_to_original(date_obj, original_format):
    if len(original_format) == 4:  # 'YYYY' 형식
        return date_obj.strftime('%Y')
    elif len(original_format) == 7:  # 'YYYY-MM' 형식
        return date_obj.strftime('%Y-%m')
    elif len(original_format) == 10:  # 'YYYY-MM-DD' 형식
        return date_obj.strftime('%Y-%m-%d')

In [20]:
# 엔티티 별 쿼리 추출을 위한 함수

# 예산
def pattern1_budget(input_time):
    
    this_month = datetime.today().strftime("%Y-%m")
    this_month = datetime.strptime(this_month, '%Y-%m')
    
    if input_time[0] < this_month:
        query = { "예외" : '과거 예산 조회가 불가능합니다' }
        return query
    elif input_time[0] > this_month:
        query = { "예산추천" : 'SELECT rp_amount FROM tb_received_paid WHERE rp_date BETWEEN DATE_ADD(NOW(), INTERVAL -3 MONTH ) AND NOW() AND rp_part = 1' }
        return query
    else:
        query = { "예산" :'SELECT uf_target_budget FROM tb_user_finance WHERE user_id = ${user_id}' }
        return query

# 예/적금
def pattern1_deposit(entity, date_query):
    query = {}
    for i in range(len(entity)):
        if entity[i][0] == "예금":
            query[f'{entity[i][0]}'] = 'SELECT rp_date, rp_detail, rp_amount FROM tb_received_paid WHERE user_id = ${user_id} AND rp_part = 1 AND rp_detail = "예금"' + " " + date_query
        elif entity[i][0] == "적금":
            query[f'{entity[i][0]}'] = 'SELECT rp_date, rp_detail, rp_amount FROM tb_received_paid WHERE user_id = ${user_id} AND rp_part = 1 AND rp_detail = "적금"' + " " + date_query
    return query

# 입/출금
def pattern1_transaction(entity, text, date_query):
    query = {}
    for i in range(len(entity)):
        if entity[i][0] == "입금":
            query[f'{entity[i][0]}'] = 'SELECT rp_date, rp_detail, rp_amount FROM tb_received_paid WHERE user_id = ${user_id} AND rp_part = 0 AND rp_detail = "입금"' + " " + date_query
        elif entity[i][0] == "출금" or "인출" in entity[i][0] or entity[i][0] == "납부":
            query[f'{entity[i][0]}'] = 'SELECT rp_date, rp_detail, rp_amount FROM tb_received_paid WHERE user_id = ${user_id} AND rp_part = 1 AND rp_detail = "출금"' + " " + date_query
        elif entity[i][0] == "입출금":
            query[f'{entity[i][0]}'] = 'SELECT rp_date, rp_detail, rp_amount FROM tb_received_paid WHERE user_id = ${user_id} AND rp_detail = "입금" OR rp_detail = "출금"' + " " + date_query
        elif entity[i][0] == "이체" or "송금" == entity[i][0]:
            if "들어온" in text or "받은" in text:
                query[f'{entity[i][0]}'] = 'SELECT rp_date, rp_detail, rp_amount FROM tb_received_paid WHERE user_id = ${user_id} AND rp_part = 0 AND rp_detail = "입금"' + " " + date_query
            elif "나간" in text or "보낸" in text:
                query[f'{entity[i][0]}'] = 'SELECT rp_date, rp_detail, rp_amount FROM tb_received_paid WHERE user_id = ${user_id} AND rp_part = 1 AND rp_detail = "출금"' + " " + date_query
            else:
                query[f'{entity[i][0]}'] = 'SELECT rp_date, rp_detail, rp_amount FROM tb_received_paid WHERE user_id = ${user_id} AND rp_detail = "입금" OR rp_detail = "출금"' + " " + date_query
    return query

# (고정)입/출금
def pattern1_transaction_fixed(entity, text, date_query):
    query = {}
    for i in range(len(entity)):
        if entity[i][0] == "입금":
            query[f'{entity[i][0]}'] = 'SELECT rp_date, rp_detail, rp_amount FROM tb_received_paid WHERE user_id = ${user_id} AND rp_part = 0 AND rp_detail = "입금" AND rp_hold = 0' + " " + date_query
        elif entity[i][0] == "출금" or "인출" in entity[i][0] or entity[i][0] == "납부":
            query[f'{entity[i][0]}'] = 'SELECT rp_date, rp_detail, rp_amount FROM tb_received_paid WHERE user_id = ${user_id} AND rp_part = 1 AND rp_detail = "출금" AND rp_hold = 0' + " " + date_query
        elif entity[i][0] == "입출금":
            query[f'{entity[i][0]}'] = 'SELECT rp_date, rp_detail, rp_amount FROM tb_received_paid WHERE user_id = ${user_id} AND (rp_detail = "입금" OR rp_detail = "출금") AND rp_hold = 0' + " " + date_query
        elif entity[i][0] == "이체" or "송금" == entity[i][0]:
            if "들어온" in text or "받은" in text:
                query[f'{entity[i][0]}'] = 'SELECT rp_date, rp_detail, rp_amount FROM tb_received_paid WHERE user_id = ${user_id} AND rp_part = 0 AND rp_detail = "입금" AND rp_hold = 0' + " " + date_query
            elif "나간" in text or "보낸" in text:
                query[f'{entity[i][0]}'] = 'SELECT rp_date, rp_detail, rp_amount FROM tb_received_paid WHERE user_id = ${user_id} AND rp_part = 1 AND rp_detail = "출금" AND rp_hold = 0' + " " + date_query
            else:
                query[f'{entity[i][0]}'] = 'SELECT rp_date, rp_detail, rp_amount FROM tb_received_paid WHERE user_id = ${user_id} AND rp_detail = "입금" OR rp_detail = "출금" AND rp_hold = 0' + " " + date_query
    return query

# 자산
def pattern1_asset(entity):
    query = {
        'query_foundation': 'SELECT uf_capital, uf_installment_saving, uf_deposit, uf_loan FROM tb_user_finance WHERE user_id = ${user_id}',
        'query_deposit': 'SELECT rp_amount FROM tb_received_paid WHERE user_id = ${user_id} AND rp_detail = "예금"',
        'query_installment_saving': 'SELECT rp_amount FROM tb_received_paid WHERE user_id = ${user_id} AND rp_detail = "적금"',
        'query_loan': 'SELECT rp_amount FROM tb_received_paid WHERE user_id = ${user_id} AND rp_detail = "대출"',
        'query_stock_qty': 'SELECT sh_ss_count, sh_ap_count, sh_bit_count FROM tb_shares_held WHERE user_id = ${user_id} ORDER BY sh_date DESC LIMIT 1',
        'query_stock_price': 'SELECT sc_ss_count, sc_ap_count, sc_bit_count FROM tb_stock ORDER BY fd_date DESC LIMIT 1',
        'query_account_plus': 'SELECT rp_amount FROM tb_received_paid WHERE user_id = ${user_id} AND rp_part = 0 AND NOT rp_detail = "대출"',
        'query_account_minus': 'SELECT rp_amount FROM tb_received_paid WHERE user_id = ${user_id} AND rp_part = 1 AND NOT rp_detail = "예금" AND NOT rp_detail = "적금"',
        'query_method': [
            '예금 = foundation.uf_deposit + query_deposit',
            '적금 = foundation.uf_installment_saving + query_installment_saving',
            '대출 = foundation.uf_loan + query_loan',
            '주식 = quary_stock_qty * query_stock_price',
            '계좌 = foundation.uf_capital + query_account_plus - query_account_minus'
        ]
    }
    return query

# 대출
def pattern1_loan(entity, text, date_query):
    query = {}
    for i in range(len(entity)):
        if "대출" and "상환" in text:
            text = text.replace("상환", "", 1)
            query['대출상환'] = 'SELECT rp_date, rp_detail, rp_amount FROM tb_received_paid WHERE user_id = ${user_id} AND rp_part = 1 AND rp_detail = "대출상환"' + " " + date_query
        else:
            query['대출'] = 'SELECT rp_date, rp_detail, rp_amount FROM tb_received_paid WHERE user_id = ${user_id} AND rp_part = 0 AND rp_detail = "대출"'
    return query

# 주식
def pattern1_stock(entity, input_date):  # [('삼성', 'STOCK'), ('애플', 'STOCK'), ('산', 'buy')]
    entity_list1 = []
    entity_list2 = []
    for i, j in entity:
        if i in ["삼성", "삼전", "samsung"]:
            stock_name = 'sh_ss_count'
            entity_list1.append(stock_name)
        elif i in ["애플", "apple"]:
            stock_name = 'sh_ap_count'
            entity_list1.append(stock_name)
        elif i in ["비트코인", "코인", "bitcoin", "coin"]:
            stock_name = 'sh_bit_count'
            entity_list1.append(stock_name)
        entity_list2.append(j)
    
    query = {}
    if "buy" in entity_list2 and "sell" in entity_list2:
        date_query = process_date_format_stock(input_date)
        print(date_query)
        if len(entity_list1) == 0:
            query['주식거래'] = 'SELECT sh_date, sh_ss_count, sh_ap_count, sh_bit_count FROM tb_shares_held WHERE user_id = ${user_id}' + date_query
        elif len(entity_list1) == 1:
            query[f'{entity_list1[0]}거래'] = f'SELECT {entity_list1[0]} FROM tb_shares_held' + ' WHERE user_id = ${user_id}' + date_query
        else:
            for i in range(len(entity_list1)):
                query[f'{entity_list1[i]}거래'] = f'SELECT {entity_list1[i]} FROM tb_shares_held' + ' WHERE user_id = ${user_id}' + date_query
        return query
    elif "buy" in entity_list2:
        date_query = process_date_format_stock(input_date)
        if len(entity_list1) == 0:
            query['주식구매'] = 'SELECT sh_date, sh_ss_count, sh_ap_count, sh_bit_count FROM tb_shares_held WHERE user_id = ${user_id} AND (sh_ss_count > 0 OR sh_ap_count > 0 OR sh_bit_count > 0)' + " " + date_query
        elif len(entity_list1) == 1:
            query[f'{entity_list1[0]}구매'] = f'SELECT {entity_list1[0]} FROM tb_shares_held' + ' WHERE user_id = ${user_id}' + f' AND {entity_list1[0]} > 0' + " " + date_query
        else:
            for i in range(len(entity_list1)):
                query[f'{entity_list1[i]}구매'] = f'SELECT {entity_list1[i]} FROM tb_shares_held' + ' WHERE user_id = ${user_id}' + f' AND {entity_list1[i]} > 0' + " " + date_query
        return query
    elif "sell" in entity_list2:
        date_query = process_date_format_stock(input_date)
        if len(entity_list1) == 0:
            query['주식판매'] = 'SELECT sh_date, sh_ss_count, sh_ap_count, sh_bit_count FROM tb_shares_held WHERE user_id = ${user_id} AND (sh_ss_count < 0 OR sh_ap_count < 0 OR sh_bit_count < 0)' + " " + date_query
        elif len(entity_list1) == 1:
            query[f'{entity_list1[0]}판매'] = f'SELECT {entity_list1[0]} FROM tb_shares_held' + ' WHERE user_id = ${user_id}' + f' AND {entity_list1[0]} < 0' + " " + date_query
        else:
            for i in range(len(entity_list1)):
                query[f'{entity_list1[i]}판매'] = f'SELECT {entity_list1[i]} FROM tb_shares_held' + ' WHERE user_id = ${user_id}' + f' AND {entity_list1[i]} < 0' + " " + date_query
        return query
    else:
        date_query = process_date_format_stock_qty(input_date)
        if len(entity_list1) == 0:
            query['주식내역'] = 'SELECT sh_date, sh_ss_count, sh_ap_count, sh_bit_count FROM tb_shares_held WHERE user_id = ${user_id}' + " " + date_query
        elif len(entity_list1) == 1:
            query[f'{entity_list1[0]}주식내역'] = f'SELECT {entity_list1[0]} FROM tb_shares_held' + ' WHERE user_id = ${user_id}' + " " + date_query
        else:
            for i in range(len(entity_list1)):
                query[f'{entity_list1[i]}주식내역'] = f'SELECT {entity_list1[i]} FROM tb_shares_held' + ' WHERE user_id = ${user_id}' + " " + date_query
        return query

# 고정
def pattern_fixed(entity1pattern, entity1, text, date_query):
    if entity1pattern == "EXPENDITURE":  # 고정지출
        quary = { "고정지출" : 'SELECT rp_date, rp_detail, rp_amount FROM tb_received_paid WHERE user_id = ${user_id} AND rp_part = 1 AND rp_hold = 0' + " " + date_query }
        return quary
    elif entity1pattern == "INCOME":  # 고정수입
        quary = { "고정수입" : 'SELECT rp_date, rp_detail, rp_amount FROM tb_received_paid WHERE user_id = ${user_id} AND rp_part = 0 AND rp_hold = 0' + " " + date_query }
        return quary
    elif entity1pattern == "TRANSACTION":  # 고정입출금
        return pattern1_transaction_fixed(entity1, text, date_query)

# 소비
def pattern1_expenditure(entity, date_query):
    query = {}
    for i in range(len(entity)):
        query[f'소비{i+1}'] = 'SELECT rp_date, rp_detail, rp_amount FROM tb_received_paid WHERE user_id = ${user_id} AND rp_part = 1' + " " + date_query
    return query

# 수입
def pattern1_income(entity, date_query):
    query = {}
    for i in range(len(entity)):
        query[f'수입{i+1}'] = 'SELECT rp_date, rp_detail, rp_amount FROM tb_received_paid WHERE user_id = ${user_id} AND rp_part = 0' + " " + date_query
    return query

# all
def pattern1_all(entity, date_query):
    query = {}
    for i in range(len(entity)):
        query[f'전체내역{i+1}'] = 'SELELT rp_date, rp_detail, rp_amount FROM tb_received_paid WHERE user_id = ${user_id}' + " " + date_query
    return query

# 구입
def pattern1_buy(entity, date_query, input_date):
    print(entity[0])
    if entity[0][0] in ["매입", "매수", "투자"]:
        return pattern1_stock(entity, input_date)
    elif entity[0][0] in ["구매", "구입", "산"]:
        return pattern1_expenditure(entity, date_query)

# 판매  판매|매도|\b판\b|처분"
def pattern1_sell(entity, date_query, input_date):
    print(entity[0])
    if entity[0][0] in ["매도", "처분", "판매", "판"]:
        return pattern1_stock(entity, input_date)
    elif entity[0][0] in ["!"]:
        return pattern1_income(entity, date_query)


In [6]:
# 앤티티 2개일때, 1개만 남기기
def filter_entities(entities):
    entity_map = {
        "ASSET": 0,             # 보유|자산|재정|재무|자본|재산|잔고
        "ALL": 0,               # 가계부 등
        "STOCK": 2,             # 주식|삼성|삼전|애플|코인|비트코
        "DEPOSIT_SAVINGS": 2,   # 적금|예금|저축
        "LOAN": 2,              # 대출
        "INCOME": 2,            # 수입|소득|월급|급여
        "BUDGET": 2,            # 예산
        "TRANSACTION": 1,       # 입출금|입금|출금|이체|송금|인출|납부
        "EXPENDITURE": 1        # 지출|소비|쓴|사용|결제|카드
    }

    vs = []
    for i in entities:
        a = {i : entity_map[i]}
        vs.append(a)

    vs2 = {}
    for i in vs:
        if len(vs2) != 0:
            ivalue = list(i.values())
            vs2value = list(vs2.values())
            if ivalue[0] > vs2value[0]:
                vs2.clear()
                vs2[list(i.keys())[0]] = list(i.values())[0]
            elif ivalue[0] == vs2value[0]:
                vs2[list(i.keys())[0]] = list(i.values())[0]
        else:
            vs2[list(i.keys())[0]] = list(i.values())[0]
    return list(vs2.keys())


In [16]:
# 쿼리 추출 함수
def create_quary(text):
    print(text)
    
    # 텍스트 전처리 함수
    text = processe_text(text)
    print("processing:", text)

    entity = extract_finance_entities(text)
    entity1 = (entity['pattern1'])
    entity2 = (entity['pattern2'])
    print("entity 패턴1:", entity1)
    print("entity 패턴2:", entity2)

    if any(keyword in text for keyword in ['앞으로', '다음 달', '다음달']):  # 예산, 고정
        input_date = ed.split_and_return_periods(text, True)
    else:            
        input_date = ed.split_and_return_periods(text, False)
    print("추출날짜:", input_date)
    
    if "고정" in text:
        if input_date == None:
            input_date = [datetime.strftime(datetime.today(), '%Y-%m')]  # 이번달

        input_date_vs, today = parse_input_date(input_date[0])
        if input_date_vs > today:
            date_list = []
            for i in input_date:
                input_date, today = parse_input_date(i)
                input_date = input_date - relativedelta(months=2)
                input_date = format_back_to_original(input_date, i)
                date_list.append(input_date)
                date_query = process_date_format(date_list)
                print("변환날짜쿼리(고정):", date_query)

        elif input_date_vs == today:
            date_list = []
            for i in input_date:
                input_date, today = parse_input_date(i)
                input_date = input_date - relativedelta(months=0)
                input_date = format_back_to_original(input_date, i)
                date_list.append(input_date)
                date_query = process_date_format(date_list)
                print("변환날짜쿼리(고정):", date_query)

        elif input_date_vs < today:
            date_query = process_date_format(input_date)
            print("변환날짜쿼리(고정):", date_query)

    entity1pattern = []
    for i in range(len(entity1)):
        entity1pattern.append(entity1[i][1])
    entity1pattern = list(set(entity1pattern))
    
    if "STOCK" in entity1pattern:
        return pattern1_stock(entity1, input_date)
    
    else:
        if len(entity1pattern) == 1:
            entity1pattern = entity1pattern[0]

            if "고정" in text:
                return pattern_fixed(entity1pattern, entity1, text, date_query)
            
            else:
                if entity1pattern == "EXPENDITURE":
                    date_query = process_month_format(input_date)
                    return pattern1_expenditure(entity1, date_query)
                
                elif entity1pattern == "INCOME":
                    date_query = process_month_format(input_date)
                    return pattern1_income(entity1, date_query)
                
                elif entity1pattern == "BUDGET":
                    if input_date == None:
                        input_date = [datetime.strftime(datetime.today(), '%Y-%m')]  # 이번달
                    input_month = find_month(i[0])
                    return pattern1_budget(input_month)
                
                elif entity1pattern == "LOAN":
                    date_query = process_month_format(input_date)
                    return pattern1_loan(entity1, text, date_query)
                
                elif entity1pattern == "DEPOSIT_SAVINGS":
                    date_query = process_all_format(input_date)
                    return pattern1_deposit(entity1, date_query)
                
                elif entity1pattern == "TRANSACTION":
                    date_query = process_month_format(input_date)
                    return pattern1_transaction(entity1, text, date_query)
                
                elif entity1pattern == "ASSET":
                    return pattern1_asset(entity1)
                
                elif entity1pattern == "ALL":
                    date_query = process_month_format(input_date)
                    print("변환날짜쿼리:", date_query)
                    return pattern1_all(entity, date_query)
                
                elif entity1pattern == "buy":
                    date_query = process_month_format(input_date)
                    return pattern1_buy(entity1, date_query, input_date)
                
                elif entity1pattern == "sell":
                    date_query = process_month_format(input_date)
                    return pattern1_sell(entity1, date_query, input_date)

        elif len(entity1pattern) == 0:
            query = { "예외": "질문에서 사용자의 의도파악에 실패하였습니다. 형식에 맞춰 다시 질문해주세요." } 
            return query
        else:
            entity1pattern = filter_entities(entity1pattern)
            print("clean엔터티1:", entity1pattern)

            if entity1pattern == "EXPENDITURE":
                    date_query = process_month_format(input_date)
                    return pattern1_expenditure(entity1, date_query)
                
            elif entity1pattern == "INCOME":
                date_query = process_month_format(input_date)
                return pattern1_income(entity1, date_query)
            
            elif entity1pattern == "BUDGET":
                if input_date == None:
                    input_date = [datetime.strftime(datetime.today(), '%Y-%m')]  # 이번달
                input_month = find_month(i[0])
                return pattern1_budget(input_month)
            
            elif entity1pattern == "LOAN":
                date_query = process_month_format(input_date)
                return pattern1_loan(entity1, text, date_query)
            
            elif entity1pattern == "DEPOSIT_SAVINGS":
                date_query = process_all_format(input_date)
                return pattern1_deposit(entity1, date_query)
            
            elif entity1pattern == "TRANSACTION":
                date_query = process_month_format(input_date)
                return pattern1_transaction(entity1, text, date_query)
            
            elif entity1pattern == "ASSET":
                return pattern1_asset(entity1)
            
            elif entity1pattern == "ALL":
                date_query = process_month_format(input_date)
                print("변환날짜쿼리:", date_query)
                return pattern1_all(entity, date_query)
            else:
                query = { "예외": "죄송합니다. 2개 이상의 재무정보는 한번에 답변이 불가능합니다. 1종류의 재무정보만 질문해주세요." }
                return query

In [8]:
def clean_query(text):
    query = create_quary(text)
    complite_query = {}
    temp_value = set()
    for i, j in query.items():
        if j not in temp_value:
            complite_query[i] = j
            temp_value.add(j)
    return complite_query

In [23]:
# rp_part = 1 지출 / rp_part = 0 입금 / rp_hold = 1
text = "오늘 이체 받은 내역 보여줘"
clean_query(text)

오늘 이체 받은 내역 보여줘
processing: 오늘 이체 받은 내역 보여줘
entity 패턴1: [('이체', 'TRANSACTION')]
entity 패턴2: [('내역', 'simple')]
추출날짜: ['2024-09-24']


{'이체': 'SELECT rp_date, rp_detail, rp_amount FROM tb_received_paid WHERE user_id = ${user_id} AND rp_part = 0 AND rp_detail = "입금" AND rp_date IN ("2024-09-24")'}

In [10]:
"""
이번 달 예상되는 지출 금액을 알려줘 = 이번달~오늘까지 사용한 금액 + 이번달 남은 기간에 고정지출
이번 달 예상되는 수입은 얼마야
다음 월급이 들어오는 날짜가 언제인지 알려줘

미래시점 단어 정리 = 다음 달 앞으로
날짜 데이터가 없을 경우

예상|계획
은행|금융|뱅킹|계좌

주식 구매/판매 내역 구할방법 ?
내역중에 특정 부분(통신비)만 보여줘 ?
자산과 주식/예금/은행 동시에 나올떄 ?

지난 달 지출 내역에서 가장 큰 항목을 알려주세요
최근 사용 내역에서 자주 결제된 항목을 확인해 주세요.
최근 결제 내역에서 식비 외의 지출을 확인해 주세요.
예금 계좌에 가장 최근에 입금한 금액 알려줘.

지출 : "고정"일 경우
대출 받을때 / 상환할때 구분
예산 : 앞시재가 과거일 경우: 불가답변, 미래일때와 과거일떄 / 설정과 사용|초과했는지
과거에 설정한 예산 조회 방법 x
주식 수익|수익율|배당|배당액 x
"""

'\n이번 달 예상되는 지출 금액을 알려줘 = 이번달~오늘까지 사용한 금액 + 이번달 남은 기간에 고정지출\n이번 달 예상되는 수입은 얼마야\n다음 월급이 들어오는 날짜가 언제인지 알려줘\n\n미래시점 단어 정리 = 다음 달 앞으로\n날짜 데이터가 없을 경우\n\n예상|계획\n은행|금융|뱅킹|계좌\n\n주식 구매/판매 내역 구할방법 ?\n내역중에 특정 부분(통신비)만 보여줘 ?\n자산과 주식/예금/은행 동시에 나올떄 ?\n\n지난 달 지출 내역에서 가장 큰 항목을 알려주세요\n최근 사용 내역에서 자주 결제된 항목을 확인해 주세요.\n최근 결제 내역에서 식비 외의 지출을 확인해 주세요.\n예금 계좌에 가장 최근에 입금한 금액 알려줘.\n\n지출 : "고정"일 경우\n대출 받을때 / 상환할때 구분\n예산 : 앞시재가 과거일 경우: 불가답변, 미래일때와 과거일떄 / 설정과 사용|초과했는지\n과거에 설정한 예산 조회 방법 x\n주식 수익|수익율|배당|배당액 x\n'